In [1]:
import pandas as pd
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")
train = pd.DataFrame(mnist.train.images)
target = pd.DataFrame(mnist.train.labels)
test = pd.DataFrame(mnist.test.images)
testLabels = pd.DataFrame(mnist.test.labels)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

Using TensorFlow backend.


In [0]:
from collections import Counter
from sklearn.metrics import confusion_matrix
import itertools
import seaborn as sns
from subprocess import check_output
%matplotlib inline

In [4]:
print(train.shape)
train.head()

(55000, 784)


,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [0]:
import io

train1 = pd.read_csv(io.BytesIO(uploaded['train.csv']))

In [7]:
train1.shape

(42000, 785)

In [0]:
#combine test and train data sets
df = pd.concat((train,test),axis=0)
df_labels = pd.concat((target,testLabels),axis=0)

In [24]:
print(df.shape)

(65000, 784)


In [0]:
train2 = train1.drop(columns='label')
labels2 = train1.label

In [26]:
train2.shape

(42000, 784)

In [0]:
import numpy as np
df2 = np.concatenate([df,train2],axis=0)
df2_labels = pd.concat((df_labels,labels2),axis=0)

In [28]:
print(df2.shape)

(107000, 784)


In [0]:
import keras
num_classes = 10
X_train1 = df2.reshape([df2.shape[0], 28, 28,1])
y_train1 = keras.utils.to_categorical(df2_labels, num_classes)

In [0]:
#X_train = train.values.reshape(train.shape[0], 28, 28,1)
#X_val = test.values.reshape(test.shape[0], 28, 28,1)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
#batch_size = 128
batch_size = 1024
epochs = 30
#epochs = 20
#epochs=17 is best 0.9965
#epochs =46 best 0.9972--bs-64
input_shape = (28, 28, 1)
#epochs = 1 # Turn epochs to 30 to get 0.9967 accuracy
#batch_size = 86
#52/60--9974--bs=50
#ep---53---9971---bs--50

In [0]:
#y_train = keras.utils.to_categorical(target, num_classes)
#y_val = keras.utils.to_categorical(testLabels, num_classes)
X_train1 = X_train1/255.0
X_train, X_val, Y_train, Y_val = train_test_split(X_train1, y_train1, test_size = 0.05, random_state=42)

In [32]:
print(X_train.shape)
print(X_val.shape)

(101650, 28, 28, 1)
(5350, 28, 28, 1)


In [0]:
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu', kernel_initializer='he_normal',input_shape = (28,28,1)))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(10, activation = "softmax"))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])
             

In [0]:
#model = Sequential()
#model.add(Conv2D(64, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal',input_shape=input_shape))
#model.add(Conv2D(64, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal'))
#model.add(MaxPool2D((2, 2)))
#model.add(Dropout(0.20))
#model.add(Conv2D(128, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
#model.add(Conv2D(128, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
#model.add(MaxPool2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
#model.add(Conv2D(256, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
#model.add(Dropout(0.25))
#model.add(Flatten())
#model.add(Dense(256, activation='relu'))
#model.add(Dense(128, activation='relu'))
#model.add(BatchNormalization())
#model.add(Dropout(0.25))
#model.add(Dense(num_classes, activation='softmax'))

#model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0),
#              metrics=['accuracy'])


In [0]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15, # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 128)       147584    
__________

In [36]:
datagen.fit(X_train)
h = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 1, steps_per_epoch=X_train.shape[0]// batch_size
                              , callbacks=[learning_rate_reduction],)

Epoch 1/30
99/99 [==============================] - 37s 377ms/step - loss: 0.0076 - acc: 0.9977 - val_loss: 0.0058 - val_acc: 0.9983
Epoch 2/30
99/99 [==============================] - 37s 372ms/step - loss: 0.0069 - acc: 0.9977 - val_loss: 0.0065 - val_acc: 0.9983
Epoch 3/30
99/99 [==============================] - 37s 370ms/step - loss: 0.0069 - acc: 0.9978 - val_loss: 0.0057 - val_acc: 0.9981
Epoch 4/30
99/99 [==============================] - 37s 370ms/step - loss: 0.0064 - acc: 0.9979 - val_loss: 0.0062 - val_acc: 0.9983

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0001.
Epoch 5/30
99/99 [==============================] - 37s 370ms/step - loss: 0.0063 - acc: 0.9980 - val_loss: 0.0059 - val_acc: 0.9983
Epoch 6/30
99/99 [==============================] - 37s 369ms/step - loss: 0.0064 - acc: 0.9980 - val_loss: 0.0057 - val_acc: 0.9981
Epoch 7/30
99/99 [==============================] - 37s 369ms/step - loss: 0.0066 - acc: 0.9979 - val_loss: 0.0059 - val_acc: 0.9985
Epo

In [37]:
final_loss, final_acc = model.evaluate(X_val, Y_val, verbose=0)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))

Final loss: 0.005120, final accuracy: 0.999065


In [38]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [0]:
import io

test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [0]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
import numpy as np
#test = pd.read_csv("../MNIST/test.csv")
test = test/255.0
test = test.values.reshape(test.shape[0], 28, 28,1)
#test = test.values.reshape(-1,28,28,1)
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [0]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_0322_9991.csv",index=False)

In [0]:
download = files.download("cnn_mnist_0322_9991.csv")